In [2]:
import meshio as mio
import meshplot as mp
import numpy as np
from utils import *
import torch
from tqdm import tqdm
import igl

In [3]:
# m = mio.read("../meshes/sphere/sphere3.msh")
m = mio.read("../meshes/armadillo/armadillo_P4_org.msh")
# m = mio.read("../meshes/microstructure/microstructure_P4.msh")

order = 3 if m.cells[-1].type == 'tetra20' else 4
T = m.cells[-1].data.astype(int)
V = m.points

In [15]:
ti = 80
# 
rV, rT = sample_tet(8)

TT = T[ti, :]
pts = V[TT]*100

target = np.array([80., 80., 70.])

opt, vv = inver_gmapping(order, rV[0, :], pts, target)



  3%|▎         | 29/1000 [00:01<00:34, 28.24it/s]


In [13]:
print(type(opt), type(pts))

print(gmapping(order, opt, pts))
print(gmapping_energy(order, opt, pts, target))

<class 'numpy.ndarray'> <class 'numpy.ndarray'>
[80.00000004 79.99999948 69.99999971]
3.6040468845961896e-13


In [14]:


# remapping = [0, 1, 2, 3,   4, 5,  6, 7,   8, 9,    11, 10,  15, 14,  13, 12,    16, 17, 19, 18]
rV, rT = sample_tet(6)

ppp = []
ttt = []
tot = 0


for ti in tqdm(range(T.shape[0])):
    TT = T[ti, :]
    pts = V[TT]

    img_p = gmapping(order, rV, pts)

    ppp.extend(img_p)
    ttt.extend(rT+tot)
    tot+=img_p.shape[0]


ppp = np.array(ppp)
ttt = np.array(ttt, dtype=int)

p_uni, _, inverse = np.unique(ppp, return_index=True, return_inverse=True, axis=0)
t_uni = np.array([inverse[ttt[:, 0]], inverse[ttt[:, 1]], inverse[ttt[:, 2]], inverse[ttt[:, 3]]]).transpose()

p = mp.plot(p_uni, t_uni, shading={"wireframe": True})
p.add_points(V, shading={"point_size": 0.05})

F = igl.boundary_facets(t_uni)

100%|██████████| 4680/4680 [00:02<00:00, 1710.67it/s]


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.4207500…

In [17]:
rV.min()

0.0

In [53]:
from scipy.spatial.transform import Rotation

In [80]:
P1 = mio.read("../meshes/armadillo/verschoor/armadillo_fine.ply")
P4 = mio.read("../meshes/armadillo/armadillo_P4_org-collision-mesh.obj")

V = P4.points.copy()

T0 = 38/30 * np.eye(3) @ Rotation.from_euler('y', -90, degrees=True).as_matrix().T
t0 = np.array([[0.485, 1.2, -0.535]])

V = V @ T0
V += t0

plt = mp.plot(P1.points, shading={"point_color": "blue", "point_size": 0.05})
plt.add_points(V, shading={"point_color": "red", "point_size": 0.05})

/home/zachary/.local/lib/python3.10/site-packages/jupyter_client/session.py:716: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 1.83…

1

In [81]:
import igl

# Use the previous cell as a good initialization
V_icp = V.copy()

R, t = igl.iterative_closest_point(
    V_icp, P4.cells[0].data.astype(int), 
    P1.points.astype(float), P1.cells[0].data.astype(int), 
    num_samples=1000, max_iters=1000)

print(Rotation.from_matrix(R).as_euler("xyz", degrees=True), t)

V_icp = V_icp @ R + t

plt = mp.plot(P1.points, shading={"point_color": "blue", "point_size": 0.05})
plt.add_points(V, shading={"point_color": "red", "point_size": 0.05})

[ 0.00111032  0.03301042 -0.03173772] [-0.00117362 -0.00054091  0.00351954]


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 1.83…

1

In [82]:
mesh = mio.read("../meshes/armadillo/armadillo_P4_org.msh")
mesh.points = (mesh.points @ T0 + t0) @ R + t

plt = mp.plot(P1.points, shading={"point_color": "blue", "point_size": 0.05})
plt.add_points(V, shading={"point_color": "red", "point_size": 0.05})
plt.add_points(mesh.points, shading={"point_color": "green", "point_size": 0.05})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 1.83…

2

In [83]:
mesh.write("../meshes/armadillo/armadillo_P4.msh", file_format="gmsh")